In [51]:
import os
import re
import zipfile
import json
import pandas as pd
import math

In [48]:
flood_db_path = 'flood_db\\gfd_v1_4'

# Loads properties for a flood event
def load_flood_properties(file):
    with zipfile.ZipFile(file) as z:
        json_file = [file_name for file_name in z.namelist() if 'json' in file_name]
        with z.open(json_file[0]) as f:
            data = f.read()
            properties = json.loads(data)
            properties['fname'] = file

    return(properties)

properties_list = [load_flood_properties(flood_db_path + '\\' + fname) for fname in os.listdir(flood_db_path)]
properties = pd.DataFrame(properties_list)
properties.to_csv('properties.tsv', sep = '\t')

In [58]:
R = 6371
cycle_days = 12
orbits_per_cycle = 175

def radius_at_lat(lat):
    return math.sin(math.pi/2 - math.radians(lat)) * R

def circumference_at_lat(lat):
    r = radius_at_lat(lat)
    return 2*math.pi*r

def prob_per_orbit(lat, swath_width):
    if (swath_width > circumference_at_lat(lat)):
        return(1)

    return swath_width / circumference_at_lat(lat)

def prob_per_orbit(lat, swath_width, flood_width=0):
    circ = circumference_at_lat(lat)
    if (circ == 0):
        return(1)
    if (swath_width + flood_width > circumference_at_lat(lat)):
        return(1)

    return (swath_width + flood_width) / circumference_at_lat(lat)

def prob_per_timeframe(lat, swath_width, flood_width=0, time=24):
    p = prob_per_orbit(lat, swath_width, flood_width)
    orbits = orbits_per_cycle / cycle_days * 24 / time
    return(1-pow(1-p, orbits))

In [59]:
swath_width = 400
properties['capture_p_in_24h'] = properties.apply(lambda x: prob_per_timeframe(x['dfo_centroid_y'], swath_width), axis=1)
properties.to_csv('properties.tsv', sep = '\t')